# LSH Algorithm Improvement By Applying Bitmap Indexing

In [4]:
import argparse
import sys
from os import listdir
from os.path import isfile, join
from typing import Dict, List, Optional, Tuple
import imagehash
from PIL import Image
import os, os.path
import cv2
from collections import Counter
import scipy as sp
import numpy as np # Import numpy library 
from sklearn.model_selection import StratifiedKFold #Import stratified kfold as we are using a 10fold cross validation approach
from skimage.feature import hog # Import Hog model to extract features
from sklearn.metrics import confusion_matrix # Import confusion matrix to evaluate the performance

In [5]:
imgs = []
labels = []
file_size = []
k = 0
path = "./data/101_ObjectCategories" # Give the dataset path here

##  Data Preprocessing:
1. Load the images using cv2
2. Image resize
3. Feature extraction: BGR to Gray conversion 
4. Feature extraction: Histogram of Oriented Gradients(HOG)

In [6]:
folder = os.listdir(path) # from the given path get the file names such as accordion, airplanes etc..
for file in folder: # for every file name in the given path go inseide that directory and get the images
    subpath = os.path.join(path,file)  # Join the name of these files to the previous path 
    
    files = os.listdir(subpath) # Take these image names to a list called files
    j = 0
    for i in range(np.size(files)): # now we shall loop through these number of files
        
        im = cv2.imread(subpath+'/'+files[0+j]) # Read the images from this subpath
        
        imgs.append(im) # append all the read images to a list called imgs
        labels.append(k) # generate a labe to every file and append it to labels list

        j += 1
        if (j == (np.size(files))):
            file_size.append(j)
   
    k += 1
     
labels = np.array(y).tolist()
ix = []
for index, item in enumerate(imgs):
    if (np.size(item) == 1):
        ix.append(index)
        del imgs[index]
        
for index, item in enumerate(y):
    for v in range(np.size(ix)):
        if (index == ix[v]):
            del y[index]
        
labels = np.array(y).astype(np.float64) 

# Function to convert an image from color to grayscale
def rgb2gray(rgb):
    gray = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)
    return gray

def resize_(image):
    u = cv2.resize(image,(256,256))
    return u

def fd_hog(image):
    fd, hog_image = hog(image, orientations=8, pixels_per_cell=(64, 64),
                    cells_per_block=(1, 1), visualize=True)
    
    return fd

a=[]
for img in imgs:
    
    b=resize_(img)
    c=rgb2gray(b)   
    d=fd_hog(c)
    a.append(d)

a=np.array(a)

## Split data
Split the data to training and validation data. We choose 70% for training and 30% for validation purposes.

In [7]:
# append 'label' and 'id' to the last two colunms
import pandas as pd
df = pd.DataFrame(a)
df['lable'] = labels
id_ = np.arange(1,len(df)+1,1)
df['id'] = id_
X = df.values

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Using PySpark to retrieve similar images

In [11]:
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
from pyspark.conf import SparkConf
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.sql import SparkSession

In [12]:
spark = SparkSession.builder \
     .master("local") \
     .appName("Image Retrieval") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()

In [13]:
Train = map(lambda x: (int(x[-1]),int(x[-2]),Vectors.dense(x[:-2])), X_train)
Train_df = spark.createDataFrame(Train,schema=['id','label',"features"])

In [14]:
Test = map(lambda x: (int(x[-1]),int(x[-2]),Vectors.dense(x[:-2])), X_test)
Test_df = spark.createDataFrame(Test,schema=['id','label',"features"])

In [15]:
brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", bucketLength=2.0,numHashTables=3)
model = brp.fit(Train_df)

In [16]:
print("The hashed dataset where hashed values are stored in the column 'hashes':")
model.transform(Train_df).show()

The hashed dataset where hashed values are stored in the column 'hashes':
+----+-----+--------------------+--------------------+
|  id|label|            features|              hashes|
+----+-----+--------------------+--------------------+
|2011|   21|[0.0,0.0,0.0,0.0,...|[[-1.0], [0.0], [...|
|3355|   44|[0.0,0.0,0.0,0.0,...|[[-1.0], [-1.0], ...|
|4986|   65|[0.22930449974017...|[[-1.0], [0.0], [...|
|4829|   64|[0.34324529743848...|[[-1.0], [0.0], [...|
|6084|   82|[0.23114755083290...|[[-1.0], [0.0], [...|
|1554|   18|[0.75324468396522...|[[-1.0], [0.0], [...|
|4903|   64|[0.12869413385510...|[[-1.0], [0.0], [...|
|3037|   39|[0.11096099499800...|[[-1.0], [0.0], [...|
|6486|   88|[0.26171495142384...|[[-1.0], [0.0], [...|
|3281|   43|[0.26141455117403...|[[-1.0], [0.0], [...|
|7416|   91|[0.22418741898694...|[[-1.0], [-1.0], ...|
|1711|   18|[0.18892780931025...|[[-1.0], [0.0], [...|
|5319|   71|[0.13585528229187...|[[-1.0], [0.0], [...|
|3902|   51|[0.12532757334717...|[[-1.0], [0.0

In [17]:
print("Approximately joining Train_df and Test_df on Euclidean distance smaller than 1:")
model.approxSimilarityJoin(Train_df, Test_df, 1.1, distCol="EuclideanDistance")\
    .select(col("datasetA.id").alias("Train_df"),
            col("datasetB.id").alias("Test_df"),
            col("EuclideanDistance")).show(30)

Approximately joining Train_df and Test_df on Euclidean distance smaller than 1:
+--------+-------+------------------+
|Train_df|Test_df| EuclideanDistance|
+--------+-------+------------------+
|     215|   2294|1.0782458048142887|
|     215|   5241|0.7923129562535386|
|     215|   5318|0.7420657232902292|
|     215|   7089|1.0916849915334395|
|     215|   7681|1.0382596097039138|
|     215|   7946|0.9005270158548503|
|     215|   8204|0.9768081522458703|
|     215|   8997|0.7807648637175937|
|     257|   3749|1.0513798620660821|
|     257|   6352|1.0308467022233077|
|     257|   8230|0.9955425834271019|
|     257|   8236|0.8690752081764956|
|     313|   2088|1.0803480397860448|
|     313|   7393|1.0644703723857685|
|     313|   8361|1.0057352131237303|
|     457|    588|0.7221171221826685|
|     457|    971|0.7414420055547025|
|     457|   1459|0.8981441513005821|
|     457|   3024|0.7316857504479307|
|     457|   4803|0.7815983736678712|
|     457|   5816| 0.613658371912618|
|     4

In [18]:
key = Vectors.dense(X_test[1][0:-2])

In [19]:
X_test[1][-2]

19.0

In [20]:
key

DenseVector([0.1597, 0.0502, 0.1769, 0.0376, 0.4228, 0.0528, 0.0885, 0.0116, 0.0922, 0.0625, 0.164, 0.1296, 0.2788, 0.1386, 0.093, 0.0413, 0.1009, 0.0821, 0.1339, 0.1331, 0.3082, 0.0976, 0.0922, 0.0519, 0.0732, 0.0174, 0.1132, 0.0857, 0.4727, 0.1279, 0.1072, 0.0027, 0.1045, 0.0936, 0.1317, 0.1251, 0.2199, 0.134, 0.1147, 0.0765, 0.1263, 0.0644, 0.0856, 0.0861, 0.1933, 0.163, 0.1624, 0.119, 0.1556, 0.1887, 0.156, 0.1087, 0.1117, 0.0686, 0.1045, 0.1062, 0.0501, 0.0504, 0.1413, 0.2452, 0.3666, 0.0595, 0.0606, 0.0262, 0.0852, 0.0366, 0.0582, 0.1093, 0.2926, 0.2241, 0.1324, 0.0616, 0.0812, 0.0528, 0.045, 0.0968, 0.1663, 0.1752, 0.2416, 0.1411, 0.129, 0.1042, 0.082, 0.1547, 0.1944, 0.0854, 0.095, 0.1552, 0.0351, 0.0451, 0.0555, 0.2875, 0.4472, 0.0482, 0.0443, 0.0372, 0.0251, 0.0124, 0.0716, 0.1633, 0.4413, 0.1745, 0.0898, 0.022, 0.0643, 0.0319, 0.0499, 0.0784, 0.2447, 0.2472, 0.1922, 0.0915, 0.1947, 0.0558, 0.054, 0.1193, 0.1361, 0.1386, 0.1436, 0.1578, 0.1023, 0.0826, 0.1048, 0.1792, 0.1848,

In [21]:
print("Approximately searching Train_df for 2 nearest neighbors of the key:")
result = model.approxNearestNeighbors(Train_df, key, 10)

Approximately searching Train_df for 2 nearest neighbors of the key:


In [22]:
result_id = result.select('id',).collect()
result_id[0].id

4920

In [23]:
result.show()

+----+-----+--------------------+--------------------+------------------+
|  id|label|            features|              hashes|           distCol|
+----+-----+--------------------+--------------------+------------------+
|4920|   65|[0.06255436828116...|[[-1.0], [-1.0], ...|0.6079006574102186|
|1879|   19|[0.02860351989748...|[[-1.0], [0.0], [...|0.6672902776895454|
|7264|   89|[0.06820392888503...|[[-1.0], [0.0], [...|0.7119586936622719|
|5054|   67|[0.08954207872490...|[[-1.0], [-1.0], ...|0.7162213043250836|
|7225|   89|[0.07672373029079...|[[-1.0], [0.0], [...|0.7331880056791682|
|3499|   47|[0.16654797336798...|[[-1.0], [0.0], [...|0.7413533417094166|
|1863|   19|[0.05745353555556...|[[-1.0], [0.0], [...|0.7422787717680772|
|1858|   19|[0.08742506093920...|[[-1.0], [0.0], [...|0.7439794764555937|
|3568|   47|[0.13329714694877...|[[-1.0], [0.0], [...|0.7450541728506829|
|7296|   90|[0.06964719750357...|[[-1.0], [-1.0], ...|0.7461979146904298|
+----+-----+--------------------+-----

In [24]:
# from matplotlib.pyplot import imshow
# imshow(imgs[4795])